# 2.0 - Motor de Recomendación Basado en Contenido (TF-IDF)

Este notebook implementa un sistema que recomienda películas similares a una dada, usando solo su sinopsis (`overview`). 

Técnicas utilizadas:
- TF-IDF (Term Frequency - Inverse Document Frequency)
- Similitud coseno


In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar dataset limpio
df = pd.read_csv('../data/clean_movies.csv')

# Nos aseguramos de que no haya valores nulos en 'overview'
df = df.dropna(subset=['overview']).reset_index(drop=True)


## 🔠 Vectorización TF-IDF

Usamos `TfidfVectorizer` de scikit-learn para convertir las sinopsis a vectores numéricos. Este paso es clave para transformar texto en datos comparables entre películas.
 

In [12]:
# Crear el modelo TF-IDF
tfidf = TfidfVectorizer(stop_words='english')

# Aplicar vectorización sobre las sinopsis
tfidf_matrix = tfidf.fit_transform(df['overview'])

print("Matriz TF-IDF generada. Dimensiones:", tfidf_matrix.shape)


Matriz TF-IDF generada. Dimensiones: (4806, 20978)


In [13]:
# Calcular la similitud entre todas las películas
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


La matriz de similitud se genera con `cosine_similarity`, que compara los vectores TF-IDF para detectar qué películas se parecen entre sí.


In [14]:
# Crear un mapeo: título en minúsculas → índice
title_indices = pd.Series(df.index, index=df['title'].str.lower())
 

In [15]:
def show_recommendations(title, df=df, sim_matrix=cosine_sim, indices=title_indices):
    title = title.lower()
    if title not in indices:
        return pd.DataFrame({'Recomendaciones': ["Película no encontrada"]})
    
    idx = indices[title]
    sim_scores = list(enumerate(sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    movie_indices = [i[0] for i in sim_scores]
    
    recs = df.iloc[movie_indices][['title', 'overview']]
    recs = recs.rename(columns={'title': '🎬 Título recomendado', 'overview': '📝 Sinopsis'})
    return recs



In [16]:
recomendaciones = show_recommendations("The Dark Knight")
print(f"🔎 {len(recomendaciones)} recomendaciones encontradas:")
recomendaciones


🔎 5 recomendaciones encontradas:


,🎬 Título recomendado,📝 Sinopsis
3,The Dark Knight Rises,Following the death of District Attorney Harve...
428,Batman Returns,"Having defeated the Joker, Batman now faces th..."
3858,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that Th...
299,Batman Forever,The Dark Knight of Gotham City confronts a das...
1360,Batman,The Dark Knight of Gotham City begins his war ...


## Conclusión

Se ha construido un motor de recomendación simple y eficiente utilizando procesamiento de texto:

- Las sinopsis fueron vectorizadas con TF-IDF.
- Se calculó la similitud entre películas usando la métrica coseno.
- Se generaron recomendaciones exitosas a partir del título ingresado.

Este sistema puede ser fácilmente ampliado para incluir otros atributos como género, actores o director.
